In [1]:
from causalinference import CausalModel
from causalinference.utils import random_data

Y, D, X = random_data()
# print(Y)
# print(D)
# print(X)
causal = CausalModel(Y, D, X)

In [2]:
print(causal.summary_stats)


Summary Statistics

                      Controls (N_c=2479)        Treated (N_t=2521)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y       -1.006        1.742        4.891        2.996        5.897

                      Controls (N_c=2479)        Treated (N_t=2521)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0       -0.332        0.976        0.338        0.960        0.692
             X1       -0.323        0.932        0.322        0.947        0.687
             X2       -0.340        0.925        0.296        0.949        0.679



In [3]:
causal.summary_stats['X_t_mean']

array([0.33848228, 0.32206083, 0.29639706])

In [4]:
causal.summary_stats.keys()

dict_keys(['N', 'K', 'N_c', 'N_t', 'Y_c_mean', 'Y_t_mean', 'Y_c_sd', 'Y_t_sd', 'rdiff', 'X_c_mean', 'X_t_mean', 'X_c_sd', 'X_t_sd', 'ndiff'])

In [5]:
causal.est_via_ols()
print(causal.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      2.974      0.034     86.755      0.000      2.907      3.042
           ATC      2.003      0.040     50.238      0.000      1.925      2.081
           ATT      3.930      0.039     99.809      0.000      3.852      4.007



C:\Users\luis_\anaconda3\lib\site-packages\causalinference\estimators\ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]


- ATE: average treatment effect
- ATC: average treatment effect for controls
- ATT: average treatment effect for treated

Constant treatment effect: $\textit{adj=1}$ in $\texttt{est_via_ols}$.

In [6]:
causal.est_propensity_s()
print(causal.propensity)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      0.055      0.036      1.545      0.122     -0.015      0.126
            X0      1.019      0.041     24.948      0.000      0.939      1.100
            X2      1.015      0.041     24.496      0.000      0.934      1.096
            X1      1.001      0.041     24.226      0.000      0.920      1.082
         X0*X1      0.106      0.040      2.630      0.009      0.027      0.185



In [7]:
causal.propensity.keys()

dict_keys(['lin', 'qua', 'coef', 'loglike', 'fitted', 'se'])

In [8]:
causal.cutoff

0.1

In [9]:
causal.trim_s()
causal.cutoff

0.09867449862777766

In [10]:
print(causal.summary_stats)


Summary Statistics

                      Controls (N_c=1998)        Treated (N_t=1997)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y       -0.560        1.418        4.004        2.199        4.564

                      Controls (N_c=1998)        Treated (N_t=1997)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0       -0.176        0.914        0.162        0.893        0.374
             X1       -0.194        0.890        0.159        0.881        0.398
             X2       -0.187        0.865        0.187        0.908        0.421



In [11]:
causal.stratify_s()
print(causal.strata)


Stratification Summary

              Propensity Score         Sample Size     Ave. Propensity   Outcome
   Stratum      Min.      Max.  Controls   Treated  Controls   Treated  Raw-diff
--------------------------------------------------------------------------------
         1     0.099     0.145       230        21     0.123     0.123     0.945
         2     0.145     0.191       202        48     0.167     0.170     1.641
         3     0.191     0.291       387       112     0.239     0.244     1.960
         4     0.292     0.343       174        76     0.317     0.318     2.307
         5     0.343     0.393       137       112     0.369     0.369     2.376
         6     0.393     0.497       258       241     0.444     0.448     2.657
         7     0.497     0.614       234       266     0.553     0.556     3.259
         8     0.614     0.711       189       310     0.660     0.664     3.683
         9     0.711     0.812       121       378     0.760     0.765     4.171
   

In [12]:
for stratum in causal.strata:
    stratum.est_via_ols(adj=1)

In [13]:
[stratum.estimates['ols']['ate'] for stratum in causal.strata]

[0.9111776691437574,
 1.6242018223515893,
 1.9124201309460178,
 2.2709272761076678,
 2.3523458243615205,
 2.6524764954649953,
 3.245230687556098,
 3.636744921531517,
 4.126494784437615,
 4.7199030098136285]

In [14]:
causal.est_via_ols()
causal.est_via_weighting()
causal.est_via_blocking()
causal.est_via_matching(bias_adj=True)
print(causal.estimates)

C:\Users\luis_\anaconda3\lib\site-packages\causalinference\estimators\weighting.py:23: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  wlscoef = np.linalg.lstsq(Z_w, Y_w)[0]



Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      2.973      0.035     84.274      0.000      2.904      3.042
           ATC      2.445      0.039     62.649      0.000      2.368      2.521
           ATT      3.501      0.039     89.940      0.000      3.425      3.577

Treatment Effect Estimates: Weighting

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      2.994      0.041     72.937      0.000      2.914      3.075

Treatment Effect Estimates: Blocking

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      2.983      0.040     75.167      0.000      2.905      3.

C:\Users\luis_\anaconda3\lib\site-packages\causalinference\estimators\matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef


In [1]:
from pyprocessmacro import Process
import pandas as pd

df = pd.read_csv("./obsdat.csv")
df.head()

,Unnamed: 0,w1,a,z,m,y,svywt,radid_person
0,2,1,1,1,1,1,0.672157,2
1,4,1,0,0,1,1,0.964990,4
2,10,1,0,0,0,0,0.672157,10
3,11,0,1,1,1,1,0.964990,11
4,12,1,1,0,0,1,0.672157,12


In [3]:
p = Process(data=df, model=13, x="a", y="y", w="w1", z="z", m="m", logit=True)
p.summary()

C:\Users\luis_\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\luis_\anaconda3\lib\site-packages\pyprocessmacro\models.py:413: RuntimeWarning: invalid value encountered in double_scalars
  coxsnell = 1 - (lnull / lmodel) ** (2 / self._n_obs)


Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 13

Variables:
    Cons = Cons
    x = a
    y = y
    w = w1
    z = z
    m1 = m

Sample size:
57727

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************


**************************************************************************
Outcome = y 
Logistic Regression Summary

       -2LL  Model LL  p-value  McFadden  Cox-Snell  Nagelkerke      n
 59357.5242 7171.6072   0.0000    0.1078        NaN         NaN  57727

Coefficients

       coeff     se       Z      p    LLCI    ULCI
Cons  0.4133 0.0251 16.4446 0.0000  0.3641  0.4626
a     0.3036 0.0371  8.1860 0.0000  0.2309  0.3763
w1   -0.2415 0.0291 -8.2855 0.0000 -0.2986 -0.1844
a*w1 -

In [5]:
ind_model = p.indirect_model

C:\Users\luis_\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
ind_model.coeff_summary()

C:\Users\luis_\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Mediator,w1,z,Effect,Boot SE,BootLLCI,BootULCI
,m,0,0,-0.006676,0.018348,-0.041534,0.029749
,m,0,1,-0.002499,0.012326,-0.026823,0.021072
,m,1,0,-0.010762,0.009678,-0.029752,0.007766
,m,1,1,-0.005191,0.011505,-0.026987,0.018216
